In [177]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [178]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import json
import time
from bs4 import BeautifulSoup

def get_links_for_cars(base_url, car_name, start_year):
    final_url = base_url + car_name + '?' + 'year' + '=' + start_year
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)
    driver.get(final_url)

    items = []

    last_height = driver.execute_script('return document.body.scrollHeight')
    itemTargetCount = 50

    while itemTargetCount > len(items):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(2)
        new_height = driver.execute_script('return document.body.scrollHeight')

        if new_height == last_height:
            break

        last_height = new_height

        elements = driver.find_elements(By.CSS_SELECTOR, 'div.bama-ad-holder')

        href_elements = []
        for element in elements:
            html_data = element.get_attribute("outerHTML")
            soup = BeautifulSoup(html_data)
            href_value = soup.find('a')['href']
            final_value = 'https://bama.ir' + href_value
            href_elements.append(final_value)
        items = href_elements


    if len(items) < 50:
        clean_links = items
    else:
        clean_links = items[0:50]

    return clean_links

In [179]:
def get_clean_page_text(url, header):
    response = requests.get(url, headers=header)
    return response.content

In [180]:
def crawler(all_urls, headers):
    key_value_map = {}
    all_cars_clean_data = []
    for url in all_urls:  
        #extract html code for each page
        new_content = get_clean_page_text(url, headers)
        soup = BeautifulSoup(new_content, 'html.parser')

        #extract price , year, title and descripton for each car
        try:
            price = float(soup.find('span',class_='bama-ad-detail-price__price').text.strip().replace(',',''))
        except:
            price = np.nan
        key_value_map['Price'] = price

        try:
            year = int(soup.find('span',class_='bama-ad-detail-title__subtitle').text.strip())
        except:
            year = np.nan
        key_value_map['Year'] = year

        try:
            title = soup.find('h1',class_='bama-ad-detail-title__title').text.strip()
        except:
            title = np.nan
        key_value_map['Title'] = title
        
        try:
            description = soup.find('div',class_='desc').text.strip()
        except:
            description = np.nan
        key_value_map['Description'] = description

        print(price, year, title)
        print(description)

        '''
        0 -> mileage
        1 -> fuel_type
        2 -> transmition_type
        3 -> body_status
        4 -> body_color
        5 -> inside_color
        '''

        other_data = soup.find_all('div',class_='bama-vehicle-detail-with-icon__detail-holder')
        
        #extract mileage value (clean value)
        try:
            mileage = int(other_data[0].text.split()[1].replace(',',''))
        except:
            mileage = np.nan
        print(mileage)
        key_value_map['Mileage'] = mileage

        #extract fuel_type value (clean value)
        try:
            text = other_data[1].text.strip()
            fuel_type = text[text.startswith('نوع سوخت') and len(' نوع سوخت'):]
        except:
            fuel_type = np.nan
        print(fuel_type)
        key_value_map['Fuel_type'] = fuel_type

        #extract transmition_type value (clean value)
        try:
            text = other_data[2].text.strip()
            transmition_type = text[text.startswith('گیربکس ') and len('گیربکس '):]
        except:
            transmition_type = np.nan
        print(transmition_type)
        key_value_map['Transmition_type'] = transmition_type

        #extract body_status value (clean value)
        try:
            text = other_data[3].text.strip()
            body_status = text[text.startswith('وضعیت بدنه ') and len('وضعیت بدنه '):]
        except:
            body_status = np.nan
        print(body_status)
        key_value_map['Body_status'] = body_status

        #extract body_color value (clean value)
        try:
            text = other_data[4].text.strip()
            body_color = text[text.startswith('رنگ بدنه ') and len('رنگ بدنه '):]
        except:
            body_color = np.nan
        print(body_color)
        key_value_map['Body_color'] = body_color

        #extract inside_color value (clean value)
        try:
            text = other_data[5].text.strip()
            inside_color = text[text.startswith('رنگ داخلی ') and len('رنگ داخلی '):]
        except:
            inside_color = np.nan
        print(inside_color)
        key_value_map['Inside_color'] = inside_color
        all_cars_clean_data.append(key_value_map)
        key_value_map = dict()
    
    return all_cars_clean_data


In [181]:
Headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'}
base_url = 'https://bama.ir/car/'
car_name = 'samand'
start_year = '1386'

clean_links = get_links_for_cars(base_url, car_name, start_year)
final_data = crawler(clean_links, Headers)
df = pd.DataFrame(final_data)
df

410000000.0 1399 سمند، LX
nan
78000
بنزینی
دنده ای
بدون رنگ
سفید
داخل کرم
380000000.0 1396 سمند، LX
بدون خرج،با دنا پلاس تیپ یک هم معاوضه میکنم
nan
بنزینی
دنده ای
بدون رنگ
سفید
داخل کرم
310000000.0 1395 سمند، LX
سند تک برگ. دیسک صفحه والوو 2 ماه تعویض-تسمه تایم تعویض- باطری 3 ماه تعویض مارک سوزوکی 72 امپر- شیشه ها دودی 40% لاستیک 205 کویر 70% - بیمه تا برج 2 سال 1403 و معاینه فنی تا برج 3 سال 1403- کفپایی سه بعدی- از لحاظ موتوری سالم و به شرط - مصرفی خودم بوده
203000
بنزینی
دنده ای
بدون رنگ
سفید
داخل قهوه ای
575000000.0 1402 سمند، سورن
◻︎◻︎ خشک … پلاکـ شده ☐☐فروش نقدی و تحویل همان لحظه یک ساعته ☐(سند ازاد) با گارانتی و بیمه شخص ثالث ✔️✔️☐☐ . 1️⃣روییت خودرو ..2️⃣تحویل خودرو … 3️⃣تسویه مبلغ … ✔️ساعت کاری ٩صبح الی ٧ عصر ☐☐.. ⚬دوستان لطفا توجه کنید ، با دلال و کاسب هیچ همکاری نمیشود⚬⚬ خودرو فقط به شخص مصرف کننده فروخته میشود⚬
nan
دوگانه سوز
دنده ای
بدون رنگ
سفید
داخل مشکی
nan 1397 سمند، LX
nan
106000
بنزینی
دنده ای
یک لکه رنگ
سفید
داخل مشکی
584000000.0 1402 سمند، سورن
دارای مانیتور
nan
بنز

,Price,Year,Title,Description,Mileage,Fuel_type,Transmition_type,Body_status,Body_color,Inside_color
0,410000000.0,1399,سمند، LX,NaN,78000.0,بنزینی,دنده ای,بدون رنگ,سفید,داخل کرم
1,380000000.0,1396,سمند، LX,بدون خرج،با دنا پلاس تیپ یک هم معاوضه میکنم,NaN,بنزینی,دنده ای,بدون رنگ,سفید,داخل کرم
2,310000000.0,1395,سمند، LX,سند تک برگ. دیسک صفحه والوو 2 ماه تعویض-تسمه ت...,203000.0,بنزینی,دنده ای,بدون رنگ,سفید,داخل قهوه ای
3,575000000.0,1402,سمند، سورن,◻︎◻︎ خشک … پلاکـ شده ☐☐فروش نقدی و تحویل همان ...,NaN,دوگانه سوز,دنده ای,بدون رنگ,سفید,داخل مشکی
4,NaN,1397,سمند، LX,NaN,106000.0,بنزینی,دنده ای,یک لکه رنگ,سفید,داخل مشکی
5,584000000.0,1402,سمند، سورن,دارای مانیتور,NaN,بنزینی,دنده ای,بدون رنگ,سفید,داخل مشکی
6,NaN,1394,سمند، سورن,سند دسته اول، مناسب برای مصرف کننده، موتور EF7...,100000.0,بنزینی,دنده ای,گلگیر تعویض,سرمه ای,داخل مشکی
7,445000000.0,1398,سمند، LX,تک برگ سند کم کارکرد دوگانه کلاس 16 بدون خط و خش,70000.0,دوگانه سوز,دنده ای,بدون رنگ,سفید,داخل کرم
8,270000000.0,1390,سمند، LX,مولتی پلاکس ، کارکرد واقعی ، ضبط پایونیرباند ک...,203000.0,بنزینی,دنده ای,دو لکه رنگ,آبی,داخل کرم
9,390000000.0,1398,سمند، LX,ماشین سمند کاملا سالم بدون ضربه شاسی سالم تخف...,112313.0,بنزینی,دنده ای,بدون رنگ,سفید,داخل خاکی


In [182]:
file_name = 'bama-cars-samand.xlsx'
# saving the excel
df = df[['Title', 'Year', 'Fuel_type', 'Transmition_type', 'Body_color', 'Body_status', 'Inside_color', 'Mileage', 'Description', 'Price']]
df.to_excel(file_name)